In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/modeljan26/ExpModel_4.hdf5
/kaggle/input/modeljan25/ExpModel_3.hdf5
/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py
/kaggle/input/modelfeb20/ExpModel_5.hdf5


In [2]:
#import warnings
#warnings.filterwarnings('ignore')

import datatable as dt
import os, gc
# import cudf
import pandas as pd
import numpy as np
# import cupy as cp
# import janestreet
#import xgboost as xgb
#from hyperopt import hp, fmin, tpe, Trials
#from hyperopt.pyll.base import scope
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
#from sklearn.model_selection import GroupKFold
#from sklearn.preprocessing import StandardScaler
#from sklearn.pipeline import Pipeline
#import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#from joblib import dump, load

import tensorflow as tf
#tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

In [3]:
SEED = 1977
tf.random.set_seed(SEED)
np.random.seed(SEED)

print('Loading...')
full_train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')

full_train_size = full_train.shape[0]
train_size = 2300000
#test_set_size = full_train_size - train_size

train = full_train.head(train_size)
print(train.shape)

#test_set = full_train.tail(test_set_size)
#print(test_set.shape)

features = [c for c in train.columns if 'feature' in c]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']

print('Filling...')
#Compute means based on the full_train set 
f_mean = full_train[features[1:]].mean()
#train = train.query('weight > 0').reset_index(drop = True)
full_train[features[1:]] = full_train[features[1:]].fillna(f_mean)
#train['action'] = (train['resp'] > 0).astype('int')

print('Saving mean...')
f_mean = f_mean.values
np.save('f_mean.npy', f_mean)

print('Finish.')

Loading...
(2300000, 138)
Filling...
Saving mean...
Finish.


In [4]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)    
        
    out = tf.keras.layers.Dense(num_labels)(x)
    #out = tf.keras.layers.Activation('linear')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.LogCosh(name='log_cosh'), 
                  metrics = tf.keras.metrics.MeanAbsolutePercentageError(name='mean_absolute_percentage_error', dtype=None
), 
                 )
    
    return model

In [5]:
batch_size = 11500
hidden_units = [80, 90, 110, 90, 80, 40]
dropout_rates = [0.2, 0.25, 0.27, 0.25, 0.2, 0.15, 0.1]
learning_rate = 1e-3

#X_tr = train.loc[:, train.columns.str.contains('feature')]
#y_tr = train.loc[:, train.columns.str.contains('resp')]

#ckp_path = f'../input/modeljan25/ExpModel_4.hdf5'
    
#model = create_mlp(X_tr.shape[1], 5, hidden_units, dropout_rates, learning_rate)
#model.summary()
    
#model.fit(X_tr, y_tr, epochs = 200, batch_size = batch_size, callbacks = None, verbose = 1)

#model.save_weights(ckp_path)

#K.clear_session()
#del model
#rubbish = gc.collect()

In [6]:
f_mean = np.load('./f_mean.npy')

In [7]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

In [8]:
model = create_mlp(130, 5, hidden_units, dropout_rates, learning_rate)
model.load_weights('../input/modelfeb20/ExpModel_5.hdf5')

In [9]:
for (test_df, pred_df) in tqdm(iter_test):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = model(x_tt, training = False).numpy().item(4)
        pred_df.action = np.where(pred > 0.001, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)